# 学習用動画データ(mp4)とアノテーションデータ(json)を読みこみ、真値を矩形で描画して動画(mp4)で保存する。

In [9]:
import cv2
import json
from PIL import Image
from PIL import ImageColor

In [10]:
# 各種パスの設定
train_movie_path = "./train_videos_resized/train_02.mp4"
train_annotations_path = "./train_annotations_resized/train_02.json"
categories_path = "./docs/categories" # カテゴリファイル。1行ごとにカテゴリ名が記載。

In [11]:
# 学習用動画データのアノテーションのロード
with open(train_annotations_path, 'r') as f:
    train_annotations_json = json.load(f)
train_annotations_json['sequence'][0]

{'Car': [{'id': 419, 'box2d': [304, 208, 325, 230]},
  {'id': 421, 'box2d': [82, 190, 104, 215]},
  {'id': 404, 'box2d': [18, 176, 62, 218]},
  {'id': 405, 'box2d': [372, 206, 415, 241]},
  {'id': 420, 'box2d': [54, 181, 85, 216]},
  {'id': 422, 'box2d': [325, 206, 337, 225]},
  {'id': 416, 'box2d': [299, 203, 305, 214]},
  {'id': 412, 'box2d': [0, 184, 18, 207]}],
 'Signal': [{'id': 15, 'box2d': [14, 26, 42, 68]},
  {'id': 12, 'box2d': [289, 157, 298, 164]}],
 'Signs': [{'id': 38, 'box2d': [269, 166, 275, 175]}],
 'Pedestrian': [{'id': 3698, 'box2d': [263, 199, 269, 226]},
  {'id': 3910, 'box2d': [270, 201, 275, 225]},
  {'id': 3941, 'box2d': [156, 188, 171, 238]},
  {'id': 4000, 'box2d': [223, 198, 229, 221]},
  {'id': 3991, 'box2d': [101, 185, 115, 242]},
  {'id': 3978, 'box2d': [147, 191, 156, 228]},
  {'id': 3986, 'box2d': [228, 198, 235, 226]},
  {'id': 3995, 'box2d': [201, 196, 208, 222]},
  {'id': 3994, 'box2d': [252, 200, 256, 220]},
  {'id': 3997, 'box2d': [189, 196, 197, 219

In [12]:
# 学習用動画データのロード
video = cv2.VideoCapture(train_movie_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))

print("画像の(横,縦)サイズ：" + str(size))
print("画像の枚数：" + str(frame_count))
print("フレームレート(fps)：" + str(frame_rate))

画像の(横,縦)サイズ：(416, 416)
画像の枚数：600
フレームレート(fps)：5


In [13]:
class CategoriesReader():
    categories = []
    categories_color = {}
    def __init__(self, categories_file_path):
        with open(categories_file_path, 'r') as f:
            categories = f.readlines()
            self.categories = [c.strip() for c in categories] # 改行文字の削除
        self.assign_colors()
    def get_categories(self):
        return self.categories

    def assign_colors(self):
        '''
        HSVの色相(H)をカテゴリ数で分割し、カテゴリごとに色を割り当てる
        '''
        cat_num = len(self.categories)
        for i, category in enumerate(self.categories):
            h = int(360 / (cat_num)) * i
            s = 100 # 100%
            v = 100 # 100%
            color = ImageColor.getrgb("hsv({},{}%,{}%)".format(str(h), str(s), str(v)))
            self.categories_color[category] = color

    def get_category_color(self, category):
        return self.categories_color[category]

# カテゴリファイルの読み込みを行う
categories = CategoriesReader(categories_path)
category_names = categories.get_categories()
print(category_names)

# 矩形描画用にカテゴリごとに色を割り当てられているのを確認する
for category in category_names:
    print("{}: RGB is {}".format(category , str(categories.get_category_color(category))))

['Car', 'Bus', 'Truck', 'Svehicle', 'Pedestrian', 'Motorbike', 'Bicycle', 'Train', 'Signal', 'Signs']
Car: RGB is (255, 0, 0)
Bus: RGB is (255, 153, 0)
Truck: RGB is (204, 255, 0)
Svehicle: RGB is (51, 255, 0)
Pedestrian: RGB is (0, 255, 102)
Motorbike: RGB is (0, 255, 255)
Bicycle: RGB is (0, 102, 255)
Train: RGB is (51, 0, 255)
Signal: RGB is (204, 0, 255)
Signs: RGB is (255, 0, 153)


In [14]:
# シーケンス番号
sequence = 0

# 動画保存用にVideoWriterオブジェクト生成
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter('./result.mp4', fmt, frame_rate, size)

while True:
    ret, frame = video.read()
    if ret is False:
        break
    # Jsonファイルからシーケンス番号に関連するカテゴリを読み込む
    for category in category_names:
        category_values = train_annotations_json['sequence'][sequence].get(category)
        if category_values is not None:
            # 矩形を検出したら
            for rect_info in category_values:
                id_number = rect_info['id']
                box2d = rect_info['box2d']
                # 画像に矩形を描画する
                pt1 = (box2d[0], box2d[1])
                pt2 = (box2d[2], box2d[3])
                color = categories.get_category_color(category)
                cv2.rectangle(frame, pt1, pt2, color, thickness=1)
                # 矩形の上にカテゴリとid番号を記述する
                # 画像，テキスト，位置（左下），フォント，スケール，色，線太さ，種類
                cv2.putText(frame, category, pt1, cv2.FONT_HERSHEY_PLAIN, 0.5, color, 1, cv2.LINE_AA)
    writer.write(frame)
    sequence += 1
writer.release()

In [15]:
video.release()
cv2.destroyAllWindows()